# Debug Plan
1.  **Analyze Crash Cause**:
    *   `InitSecurityHooks` runs in `initialize` (constructor).
    *   It calls `ActivateBypass` and `HOOK`, which use `getRealOffset`.
    *   `getRealOffset` searches dependent libraries (UnityFramework).
    *   At constructor time, `UnityFramework` might not be loaded yet.
    *   Result: `getRealOffset` returns 0 (or Main base). Writing to 0 or wrong base -> Crash.
    *   Also, `0x1048041B8` implies a base of `0x100000000`, which is incorrect for a relative offset addition if the base is effectively added twice or mismatched.

2.  **Proposed Fix**:
    *   Delay Game-Specific Hooks until `UnityFramework` is detected.
    *   Fix the `0x1048041B8` offset to `0x48041B8`.
    *   Split `InitSecurityHooks` into `InitSystemHooks` (safe, immediate) and `InitGameHooks` (delayed).

3.  **Action Steps**:
    *   Modify `savagexiter.mm`:
        *   Wait for Unity in `hack_thread`.
        *   Move `ActivateBypass` and `HOOK(0x52AC18)` to the delayed execution path.
        *   Update `hooking` offset.
    *   Build and Deploy.
